In [ ]:
import pandas as pd
import altair as alt
import numpy as np

In [ ]:
atg_scores = '../Data/ATG_lib_data/20250409_BARD1_X1A_ATG_scored.xlsx'

In [ ]:
def read_scores(file):
    df = pd.read_excel(file)
    df = df[['pos', 'allele', 'canonical_start', 'start_pos_id', 'Consequence', 'AAsub', 'score']]

    return df

In [ ]:
def process_df(df):
    df['cds'] = df['pos'].transform(lambda x: 214809490 - (214809410 - (214809490 - x)))

    rev_comp_dict = {'A': 'T', 
                     'C': 'G', 
                     'T': 'A',
                     'G': 'C'
                    }
    df['sense_allele'] = df['allele'].transform(lambda x: rev_comp_dict[x])
    
    return df

In [ ]:
def heatmap(df):

    starts = ['ATG', 'CTG', 'AGG', 'TAG', 'ATA', 'GTG', 'ACG']

    all_maps = []
    for start in starts:
        current_df = df.copy()
        current_df = current_df.loc[current_df['canonical_start'].isin([start])]

        file_name = '/Users/ivan/Desktop/ATGlib_maps/' +'BARD1_X1A_ATG_' + start + 'map.png'
        
        #print(file_name)
        map = alt.Chart(current_df).mark_rect(strokeWidth = 7.5, width = 50, height = 50).encode(
            x = alt.X('cds:O',
                     axis = alt.Axis(
                         orient = 'top',
                         labelAngle = 0,
                         labelFontSize = 20,
                         domain = False,
                         ticks = False,
                         labelPadding = 15
                     )
                     ),
            y = alt.Y('allele:O',
                     axis = alt.Axis(title = '',
                                     labelFontSize = 20,
                                     domain = False,
                                     ticks = False,
                                     labelPadding = 15
                                    )
                     ),
            color = alt.Color('max(score)',
                              scale = alt.Scale(
                                  domain = [-0.3, 0],
                                  scheme = 'magma'
                              ),
                              title = 'SGE Score'
                             ),
            stroke = alt.Stroke('Consequence', 
                                scale = alt.Scale(
                                    scheme = 'set1'
                                ),
                                sort = ['stop_gained', 'synonymous_variant', 'missense_variant'],
                                legend = alt.Legend(
                                    symbolFillColor = 'white',
                                    labelFontSize = 12,
                                    titleFontSize = 12
                                )
                               )
        ).properties(
            width = 400,
            height = 200,
            title = start + ' map'
        ).configure_view(
            strokeWidth = 0
        )

        map.save(file_name, ppi = 500)
        map.display()

In [ ]:
def main():
    raw_df = read_scores(atg_scores)
    final_df = process_df(raw_df)
    heatmap(final_df)

In [ ]:
main()